In [1]:
# function definition
import cv2 as cv
import numpy as np
from PIL import Image

def blend_and_color_masks(segmentation_mask, gt_mask):
    """
    Merge the segmentation mask and GT mask, coloring the segmentation as (0, 128, 0),
    GT as (128, 0, 0), and combining them with 0.5 alpha blending.

    Args:
        segmentation_mask (np.ndarray): Grayscale segmentation mask.
        gt_mask (np.ndarray): Grayscale GT mask.

    Returns:
        np.ndarray: Colorized and merged mask as an RGB image.
    """
    # Create an RGB image to hold the result
    segmentation_rgb = np.zeros((*segmentation_mask.shape, 3), dtype=np.uint8)
    gt_rgb = np.zeros((*gt_mask.shape, 3), dtype=np.uint8)

    # Color the segmentation mask in green (0, 128, 0)
    segmentation_area = segmentation_mask > 0
    segmentation_rgb[segmentation_area] = [0, 128, 0]

    # Color the GT mask in red (128, 0, 0)
    gt_area = gt_mask > 0
    gt_rgb[gt_area] = [0, 0, 128]

    # Blend the two masks with 0.5 alpha
    merged_mask = cv.addWeighted(segmentation_rgb, 0.5, gt_rgb, 0.5, 0)

    return merged_mask

def merge_and_color_masks(segmentation_mask, gt_mask):
    """
    Merge the segmentation mask and GT mask, coloring the segmentation as (0, 128, 0),
    GT as (128, 0, 0), and the intersection as white (255, 255, 255).

    Args:
        segmentation_mask (np.ndarray): Grayscale segmentation mask.
        gt_mask (np.ndarray): Grayscale GT mask.

    Returns:
        np.ndarray: Colorized and merged mask as an RGB image.
    """
    # Create an RGB image to hold the result
    merged_mask = np.zeros((*segmentation_mask.shape, 3), dtype=np.uint8)

    # Color the segmentation mask in green (0, 128, 0)
    segmentation_area = segmentation_mask > 0
    merged_mask[segmentation_area] = [0, 128, 0]

    # Color the GT mask in red (128, 0, 0)
    gt_area = gt_mask > 0
    merged_mask[gt_area] = [0, 0, 128]

    # Color the intersection in white (255, 255, 255)
    intersection = segmentation_area & gt_area
    merged_mask[intersection] = [255, 255, 255]

    return merged_mask

def equalize_histogram(image):
    """
    Equalizes the histogram of the input image
    :param image: The uint16 image to equalize
    :out: The equalized image
    """
    hist, bins = np.histogram(image.flatten(), bins=65536, range=[1, 65534])
    # Calculate the cumulative distribution function (CDF)
    cdf = hist.cumsum()
    cdf_normalized = cdf / cdf[-1]
    # Use the CDF to map the original image values to the equalized values
    equalized = np.interp(image.flatten(), bins[:-1], cdf_normalized * 65535)
    # Reshape back to the original image shape and cast to uint16
    equalized_histogram = equalized.reshape(image.shape).astype(np.uint16)
    return equalized_histogram


JET_COLORMAP = np.array([
        [0.5, 0, 0],     # Dark red     # further away
        [1, 0, 0],       # Red
        [1, 0.5, 0],     # Orange
        [1, 1, 0],       # Yellow
        [0.5, 1, 0.5],   # Green
        [0, 1, 1],       # Light cyan
        [0, 0.5, 1],     # Cyan
        [0, 0, 1],       # Blue
        [0, 0, 0.5],     # Dark blue    # closer
], dtype=np.float64)
DISTANCE_COLORMAP = np.array([
    [1.0, 0.0, 0.0],  # further away
    [0.5, 0.5, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.5, 0.5],
    [0.0, 0.0, 1.0],  # closer
], dtype=np.float64)
def depth_to_colormap(depth_map, colormap='jet', equalize=True):
    """
    Uses the depth map to generate a jet map
    :param depth_map: The input depth map
    :out: The computed jet map
    """
    # Determine the colormap to use
    if colormap.lower() == 'jet':
        colormap = JET_COLORMAP
    elif colormap.lower() == 'distance':
        colormap = DISTANCE_COLORMAP
    else:
        raise ValueError(f"Unsupported colormap: {colormap}")
    # Equalize the histogram of the depth image
    if equalize:
        depth_map = equalize_histogram(depth_map)
    # Normalize the image to [0, 1]
    normalized_image = depth_map.astype(np.float64) / 65535.0
    # 0 values to Null
    normalized_image[normalized_image <= 0.001] = np.nan
    # Generate interpolated colormap values
    indices = np.linspace(0, 1, len(colormap))
    red_interp = np.interp(normalized_image.flat, indices, colormap[:, 0]).reshape(depth_map.shape)
    red_interp = np.nan_to_num(red_interp, 0)
    green_interp = np.interp(normalized_image.flat, indices, colormap[:, 1]).reshape(depth_map.shape)
    green_interp = np.nan_to_num(green_interp, 0)
    blue_interp = np.interp(normalized_image.flat, indices, colormap[:, 2]).reshape(depth_map.shape)
    blue_interp = np.nan_to_num(blue_interp, 0)
    # Combine channels and scale back to uint16 range [0, 65535]
    colormapped_image = np.stack([
        (red_interp * 65535).astype(np.uint16),
        (green_interp * 65535).astype(np.uint16),
        (blue_interp * 65535).astype(np.uint16)
    ], axis=-1)
    return colormapped_image

def superpose_map(og_image, map_image, alpha=0.5):
    """
    Superposes the map on the original image
    :param og_image: The original image
    :param map_image: The map image
    :param alpha: The alpha value to use
    :out: The superposed image
    """
    # Superpose the map on the original image
    superposed_image = cv.addWeighted(og_image, 1.0, map_image, alpha, 0)
    return superposed_image

def depth_to_normal(depth_map, equalize=True):
    """
    Uses the depth map to generate a normal map
    :param depth_map: The input depth map
    :out: The computed normal map
    """
    if len(depth_map.shape) != 2:
        raise ValueError("The input depth map must be a single channel image")
    if equalize:
        depth_map = equalize_histogram(depth_map)

    if len(depth_map.shape) != 2:
        depth_map = depth_map[:, :, 0]
    
    # set every 0 value to null value
    depth_map = depth_map.astype(np.float32)
    depth_map[depth_map == 0] = np.nan
    
    # now compute the partial derivatives of depth respect to x and y
    dx = cv.Sobel(depth_map, cv.CV_32F, 1, 0)
    dy = cv.Sobel(depth_map, cv.CV_32F, 0, 1)

    # compute the normal vector per each pixel
    normal = np.dstack((-dx, -dy, np.ones(depth_map.shape)))
    norm = np.sqrt(np.sum(normal**2, axis=2, keepdims=True))
    normal = np.divide(normal, norm, out=np.zeros_like(normal), where=norm!=0)

    # map these normal vectors to 65535
    normal = (normal + 1) * 32767.5
    # set every nan value to 0
    normal[np.isnan(normal)] = 0
    normal = normal.clip(0, 65535).astype(np.uint16)

    
    # return the map
    return normal



def compute_iou(segmentation_path, ground_truth_path):
    """
    Computes the Intersection over Union (IoU) of a segmentation mask and ground truth.
    
    Args:
        segmentation_path (str): Path to the segmentation mask image (RGB).
        ground_truth_path (str): Path to the ground truth image (RGB).
        
    Returns:
        float: The IoU value.
    """
    # Load images
    segmentation = np.array(Image.open(segmentation_path).convert("RGB"))
    ground_truth = np.array(Image.open(ground_truth_path).convert("RGB"))
    
    # Convert to binary masks
    segmentation_binary = (segmentation > 0).any(axis=-1).astype(np.uint8)
    ground_truth_binary = (ground_truth > 0).any(axis=-1).astype(np.uint8)
    
    # Compute intersection and union
    intersection = np.logical_and(segmentation_binary, ground_truth_binary).sum()
    union = np.logical_or(segmentation_binary, ground_truth_binary).sum()
    
    # Calculate IoU
    iou = intersection / union if union != 0 else 0.0
    return iou

In [54]:
# load the images
import cv2 as cv
import numpy as np
import os.path as osp
BASE_DIR = '/media/jorge/HDD/TFG/report/media/masks/'
gt_mask_path = osp.join(BASE_DIR, '2_cmx_jet_moddrop_gt.png')
ird_mask_path = osp.join(BASE_DIR, '2_cmx_jet_moddrop_ird.png')
ir_mask_path = osp.join(BASE_DIR, '2_cmx_jet_moddrop_ir.png')
d_mask_path = osp.join(BASE_DIR, '2_cmx_jet_moddrop_d.png')

gt_mask = cv.imread(gt_mask_path, cv.IMREAD_GRAYSCALE)
ird_mask = cv.imread(ird_mask_path, cv.IMREAD_GRAYSCALE)
ir_mask = cv.imread(ir_mask_path, cv.IMREAD_GRAYSCALE)
d_mask = cv.imread(d_mask_path, cv.IMREAD_GRAYSCALE)

# merge the masks
superposed_ird = merge_and_color_masks(ird_mask, gt_mask)
superposed_ir = merge_and_color_masks(ir_mask, gt_mask)
superposed_d = merge_and_color_masks(d_mask, gt_mask)

cv.imwrite('superposed_ird.png', superposed_ird)
cv.imwrite('superposed_ir.png', superposed_ir)
cv.imwrite('superposed_d.png', superposed_d)

True

In [13]:
# colormap images
import os
os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"
import cv2 as cv

image_path_1 = '/media/jorge/HDD/TFG/data/lindenthal-camera-traps/lindenthal_coco/images/20200830035607/depth/000071.exr'
image_path_2 = '/media/jorge/HDD/TFG/data/lindenthal-camera-traps/lindenthal_coco/images/20200807023726/depth/000101.exr'
image_path_3 = '/media/jorge/HDD/TFG/data/lindenthal-camera-traps/lindenthal_coco/images/20200807031150/depth/000071.exr'
image_path_4 = '/media/jorge/HDD/TFG/data/lindenthal-camera-traps/lindenthal_coco/images/20200807061001/depth/000041.exr'
image_paths = [image_path_1, image_path_2, image_path_3, image_path_4]
image_paths = ['/media/jorge/HDD/TFG/report/media/masks/difficult_depth_2.png', '/media/jorge/HDD/TFG/report/media/masks/rgb_depth.png',
               '/media/jorge/HDD/TFG/report/media/masks/rgb_depth_2.png']

for i, fpath in enumerate(image_paths):
    depth_map = cv.imread(fpath, cv.IMREAD_ANYCOLOR | cv.IMREAD_ANYDEPTH)
            # map the image to 65535 by looking at the maximum value
    depth_map = (depth_map / depth_map.max() * 65535).astype(np.uint16)
    colormap_jet = depth_to_colormap(depth_map, colormap='jet', equalize=True)
    print(cv.imwrite(fpath.replace('depth', 'jet'), colormap_jet))

True
True
True


In [16]:
# superponer dos imagenes
import cv2 as cv

image_path_1 = '/media/jorge/HDD/TFG/report/media/data/ir_gueese.jpg'
image_path_2 = '/media/jorge/HDD/TFG/report/media/data/mask_gueese.png'

image_1 = cv.imread(image_path_1)
image_2 = cv.imread(image_path_2)

out = superpose_map(image_1, image_2, alpha=0.5)

cv.imwrite('/media/jorge/HDD/TFG/report/media/data/superposed_gueese.png', out)

True

In [ ]:
# apply distance colormap to image
import cv2 as cv
image_path = '/media/jorge/HDD/TFG/report/media/data/depth_gueese.png'
image = cv.imread(image_path, cv.IMREAD_ANYCOLOR | cv.IMREAD_ANYDEPTH)
image = (image / image.max() * 65535).astype(np.uint16)
colormap_distance = depth_to_normal(image, equalize=False)
cv.imwrite(image_path.replace('depth', 'normals'), colormap_distance)

True

In [31]:
# apply depth
import json
import cv2 as cv
import numpy as np
# add src to path
import sys
sys.path.append('/media/jorge/HDD/TFG')
from src.depth2hha.getHHA import getHHA

image_path = '/media/jorge/HDD/TFG/report/media/data/depth_gueese.png'
intrinsics_path = '/media/jorge/HDD/TFG/data/lindenthal-camera-traps/lindenthal_coco/images/20200807015315/intrinsics.json'
image = cv.imread(image_path, cv.IMREAD_ANYCOLOR | cv.IMREAD_ANYDEPTH)
image = (image / image.max() * 65535).astype(np.uint16)
with open(intrinsics_path, 'r') as intrinsics_file:
    intrinsics = json.load(intrinsics_file)
cam_matrix = np.matrix([
    [intrinsics['fx'],  0,                  intrinsics['ppx']],
    [0,                 intrinsics['fy'],   intrinsics['ppy']],
    [0,                 0,                  1]
])
hha_frame = getHHA(cam_matrix, image)
cv.imwrite('hha.png', hha_frame)


True

In [17]:
# Example usage
segmentation_path = "/media/jorge/HDD/TFG/report/media/masks/2_cmx_jet_moddrop_d.png"
ground_truth_path = "/media/jorge/HDD/TFG/out/log_lindenthal-camera-traps_mit_b0_pretrained_jet_dropout/generationRGB/20200807023726/color/000101_gt.png"

iou = compute_iou(segmentation_path, ground_truth_path)
print(f"IoU: {iou:.4f}")

IoU: 0.6194
